<a href="https://colab.research.google.com/github/shIsmael/DeepLearning/blob/main/Deep_Learning_Logistic_Regression_with_a_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instructions:
1. Make a copy of this notebook by opening the "File" tab and selecting "Save a copy in Drive"
2. Close this tab and move to your copy of this notebook
3. Follow the written guides within this notebook
4. If instructed to, add your own code in the corresponding cell

After completing this notebook, you will have:
- An understanding of what logistic regression is, and how it can be applied to binary classification problems
- Built a simple neural network to model a logistic regression algorithm
- Learned how to minimize a cost function
- An understanding of how derivatives of the cost function are used to update parameters



### Binary Classification and Notation
Let's start with a binary classification problem, to which we can apply logistic regression: given an input image $x$, output a label that classifies the image as being of a cat or a non-cat, with 1 representing a cat and 0 representing a non-cat. We'll be using a dataset comprised of color images, with the dimensions of each image being 64 x 64. This dataset will be split into a training set (used for training the logistic regression algorithm) and a testing set (used for testing our algorithm's classification accuracy). 

Therefore, the input to our logistic regression algorithm (which will be explained shortly) will be 3 matrices with shape (64, 64), which totals to a input shape of (64, 64, 3); The 3 matrices represent the red, green, and blue channels that comprise a color image. We'll "flatten" the input shape of (128, 128, 3) to be a 12288-dimensional column vector (in other words, our input will be a column vector with 12288 values, since 64\*64\*3 = 12288). The dimension of this column vector, representing a given training example, will be called $n_x$. 

As for notation, an example at index $i$ in the dataset will be represented with a pair $(x^{(i)}, y^{(i)})$, where $x$ is the input image represented by a vector $x\in \mathbb{R}^{n_x}$ (a vector of with $n_x$ values) and $y$ is the output label, represented by either 0 or 1. The length of the dataset is denoted by $m$. The number of examples in the training set will be denoted with $M_{train}$, and the number of examples in the testing set will be denoted by $M_{test}$. 

To use a more compact notation to denote all of the inputs from our dataset, we'll use $X$ to represent a matrix containing every input column vector, from $x^{(1)}$ to $x^{(m)}$. This matrix will have $m$ columns and $n_x$ rows. Similarly, we'll define every output (label) from our dataset as $Y$, a matrix containg every label from $y^{(1)}$ to $y^{(m)}$. The shape of this matrix will be (1, m).

Of course, you don't need to memorize the notation piled in this cell! It's fine if you need to refer back to this cell/the notation sheet later in this notebook or during quizzes. 

In [ ]:
# importing the necessary libraries
import numpy as np # you know what this is
import matplotlib.pyplot as plt # for plotting graphs
import h5py # for working with datasets stored on H5 files
import scipy # for testing
from PIL import Image # for testing
from scipy import ndimage # for testing
%matplotlib inline 

# download the datasets from  https://drive.google.com/drive/folders/1R5kzlNNvhABEm2oCpxwXsmmV48Vp1kQQ?usp=sharing and import them into this notebook when prompted
# don't worry about understanding this code; press "cancel upload" if you wish to run this cell again
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn]))) 

# defining a function to load both the train and test datasets; you should be able to see what's happening in this function if you recall the numpy tutorial
def load_dataset():
    train_dataset = h5py.File('train_catvnoncat.h5', "r") # loading the train H5 file
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_catvnoncat.h5', "r") # loading the test H5 file
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes # "_orig" is used because we are going to preprocess these shortly

# loading the dataset into multiple variables
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()
# each line of train/test_set_x_orig is array representing an image

index = 128 # arbitrary number, set this to anything to view an image at a given index
# viewing a cat image for fun! don't worry about understanding this code
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:, index]) + ", it's a " + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  " picture")

# various statistics that are useful to know in relation to the concepts defined in the above cell
# being able to verify the dimensions of matrices/vector is also important for avoiding bugs later on 
m_train = train_set_x_orig.shape[0] # as you can see in the below print statements, we're simply accessing the first dimension of the shape of the training set
m_test = test_set_x_orig.shape[0] 
print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test)) # we'll cover how datasets should be partitioned in chapter 5! 
print ("Height/Width of each image: num_px = " + str(test_set_x_orig.shape[1]))
print ("Each image is of size: (" + str(test_set_x_orig.shape[1]) + ", " + str(test_set_x_orig.shape[1]) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape)) 
print ("test_set_y shape: " + str(test_set_y.shape))

# we can now proceed to "flatten" our training and test input datasets into datasets containing (12288, 1) vectors
# don't worry about the actual reshaping operation if you can't comprehend why it works
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T # reshaping images of shape (num_px, num_px, 3) into shape (num_px*num_px*3, 1)
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1).T
print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape)) # 209 represents the number of examples in the training dataset
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))

# one common preprocessing step in machine learning is to center and standardize your dataset, meaning that you substract the mean of the whole 
# array from each example, and then divide each example by the standard deviation of the whole numpy array; but for picture datasets, it is 
# simpler and more convenient to just divide every row of the dataset by 255, the maximum value of a pixel channel
train_set_x = train_set_x_flatten/255
test_set_x = test_set_x_flatten/255

# and with this, we've successfully loaded and preprocessed our dataset!

### Logistic Regression
Logistic regression is a learning algorithm used for binary classification problems (i.e. when the output label can be either 0 or 1). In the context of the problem stated above, we need to use logistic regression (implemented as a very small neural network) to output a prediction $\hat{y} = P(y=1|x)$ given an input image $x$. The parameters of our logistic regression algorithm will be a vector $w\in\mathbb{R}^{n_x}$ (a vector with $n_x$ dimensions) which we'll call the "weights", and a real number $b\in\mathbb{R}$ ($\mathbb{R}$ is the set of all real numbers!), which we'll call the "bias". 

Given these parameters, we can construct an initial linear function: $\hat{y}^{(i)} = w^{T}x^{(i)} + b$ (we're transposing our weight vector $w$ to make matrix multiplication possible!). Since we want to our prediction to be in the range [0,1], we wrap a sigmoid function around the entire function to obtain our equation for logistic regression: $\hat{y}^{(i)} = \sigma(w^{T}x^{(i)} + b)$, as the sigmoid function can "squish" any input into the range [0,1]; Positive inputs will be closer to 1, while negative inputs will be closer to 0. To simplify our notation, we'll denote $w^{T}x^{(i)} + b$ as $z^{(i)}$, so the equation for prediction on example $i$ is $\hat{y}^{(i)} = \sigma(z^{(i)})$.

In [ ]:
# we'll implement some functions to help us in defining the basic logistic regression neural network later

# sigmoid!
def sigmoid(z):
    s = 1/(1 + np.exp(-z))
    return s

# this function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0
def initialize_with_zeros(dim): # dim is the size of the w vector we want (or number of parameters, in this case)
    w = np.zeros((dim, 1)) 
    b = 0 
    assert(w.shape == (dim, 1)) # checking w's shape with the assert function, which will cause our program to halt if the assertion is not true
    assert(isinstance(b, float) or isinstance(b, int)) 
    
    return w, b 

### Gradient Descent
To iteratively train (adjust) the parameters $w$ and $b$, we need to define a cost function, which is the average value of a loss function over all training examples. We need our loss function to get closer to 0 as our prediction gets closer to the true label, so we'll use the following equation: $\mathcal{L}(\hat{y}, y) = -(y\:log\:\hat{y}+(1-y)log(1-\hat{y}))$. If $y=1$ for a given example, the loss function reduces to $\mathcal{L}(\hat{y}, y) = -log\:\hat{y}$, which we want to minimize; The reduced loss function can only be zero if $\hat{y} = 1$, since $log(1) = 0$. Similarly, we can reduce the loss function to $\mathcal{L}(\hat{y}, y) = -log(1-\hat{y})$ if $y=0$, which we want to minimize; The reduced loss function can only be zero if $\hat{y} = 0$.

Now that we've defined the loss function (which is applied to a single training example), we can define the cost function as the average of the loss function over all training examples: $J(w,b) = \frac{1}{m}\sum^{m}_{i=1} \mathcal{L}(\hat{y}^{(i)},y^{(i)})$. The cost function represents the "cost" of the parameters $w$ and $b$. 

We can now use the gradient descent algorithm to find the most optimal values for our parameters such that the cost function $J(w,b)$ is minimized. Luckily, our cost function $J$ is a convex function, meaning that if we plot our function, it resembles a big bowl in 3-dimensional space. This makes optimization much easier, since we don't have to worry about getting stuck in local optima (an optimization algorithm may get stuck in a "bowl" in the cost function that appears to be the global optimum, but is really not). 

To start, we can choose random values for $w$ and $b$/initialize our parameters to 0 (it doesn't matter where we start). We can then iteratively update our parameters with the following equations: $w := w - \alpha\frac{\partial J}{\partial w}$ and $b := b - \alpha\frac{\partial J}{\partial b}$, where ":=" represents updating a value with the value on the right side.. These two equations intuitively represent updating $w$ or $b$ by the negative of the partial derivative of the cost function with respect to $w$ or $b$, with $\alpha$ controlling the magnitude of the update (if we set it too high, we might overshoot the global optimum!). Recall from the calculus tutorial the fact that the gradient points in the direction of steepest ascent. By using the negative of the partial derivative of $J$, we can move in the direction of steepest *descent*, which we will use to reach the global minimum of the cost function $J$. 

In [ ]:
# defining a propagation algorithm that propagates an input forward to compute the prediction and the necessary gradient for optimization
def propagate(w, b, X, Y): 
    # X is data of shape (num_px * num_px * 3, number of examples)
    # Y is the true label vector of shape (1, number of examples)
    
    m = X.shape[1]
    
    # computing the prediction as defined in "Logistic Regression"
    Z = np.dot(w.T,X)+b
    A = 1/(1 + np.exp(-Z))               
    cost = -1*((np.sum((Y*np.log(A))+((1-Y)*(np.log(1-A))),axis=1))/m) # computing the cost, as defined above
    
    # finding the necessary partial derivatives, don't worry about the actual computation
    dw = np.dot(X,(A-Y).T)/m
    db = np.sum((A-Y),axis=1)/m
    assert(dw.shape == w.shape) # sanity checks
    assert(db.dtype == float)
    cost = np.squeeze(cost) 
    assert(cost.shape == ())
    
    grads = {"dw": dw, # gradient of the loss with respect to w (same shape as w)
             "db": db} # gradient of the loss with respect to b (same shape as b) 
    
    return grads, cost # cost is the negative log-likelihood cost defined in the above cell

# defining our gradient descent algorithm
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    # X is data of shape (num_px * num_px * 3, number of examples)
    # Y is the true label vector of shape (1, number of examples)
    # num_iterations is the number of iterations of the optimization loop

    costs = []
    
    for i in range(num_iterations):
        # cost and gradient calculation
        grads, cost = propagate(w, b, X, Y)
        
        # retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule, as defined above
        w = w - learning_rate*dw
        b = b - learning_rate*db
        
        # recording the costs
        if i % 100 == 0:
            costs.append(cost) # list of all the costs computed during the optimization, used to plot the learning curve
        
        # prints the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost)) # if print_cost was true, print the loss every 100 steps
    
    params = {"w": w, # dictionary containing the weights w and bias b
              "b": b}
    
    grads = {"dw": dw, # dictionary containing the gradients of the weights and bias with respect to the cost function
             "db": db}
    
    return params, grads, costs

# using w and b to implement a prediction algorithm on dataset X
def predict(w, b, X):  
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # implementing the prediction equations as defined in "Logistic Regression"
    Z = np.dot(w.T,X)+b
    A = 1/(1 + np.exp(-Z)) 
    
    for i in range(A.shape[1]):
        # converting probabilities A[0,i] to actual predictions p[0,i], since sigmoid can output values between 0 and 1
        if A[0,i] > 0.5:
            Y_prediction[0,i] = 1
        else:
            Y_prediction[0,i] = 0
    
    assert(Y_prediction.shape == (1, m)) # sanity check
    
    return Y_prediction #  a numpy array (vector) containing all predictions (0 or 1) for every in example in X

### Putting It All Together
Here, we put together everything covered in the past few cells into one program that defines, trains, and tests our neural network for our logistic regression problem. We'll initialize our parameters, optimize for 2000 iterations, and make predictions for testing. 

In [ ]:

def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = True):

    """
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model
    """
    
    # initialize parameters with zeros 
    w, b = initialize_with_zeros(12288)

    # gradient descent for 2000 iterations with a learning rate of 0.5 (the learning rate is an example of a hyperparameter, which we'll learn how to adjust later)
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost = True)
    
    # retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    # use parameters to predict test/train set examples 
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    # print train/test errors, don't worry if you don't know how to do this
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True) # this runs the above function

# as you can see, the training and test accuracies are fairly decent